In [1]:
# импортируем необходимые библиотеки, классы и функции
import pandas as pd
import numpy as np
import pathlib
from tqdm.notebook import tqdm

from etna.datasets import TSDataset
from etna.pipeline import Pipeline
from etna.metrics import MSE
from etna.transforms import (LagTransform,
                             MinMaxScalerTransform,
                             MeanTransform,
                             DateFlagsTransform,
                             TimeSeriesImputerTransform,
                             FilterFeaturesTransform)

from etna_utils import LGBMMultiSegmentModel

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.9/site-packages/etna/settings.py:61: UserWarning: `tsfresh` is not available, to install it, run `pip install tsfresh==0.19.0 && pip install protobuf==3.20.1`
  warnings.warn(


# Загрузка данных

In [2]:
# задаем константную часть пути
DATA_PREFIX = pathlib.PurePath('/Users/artemgruzdev/Documents/GitHub/'
                               'Time_Series/Code/Data/store_sales')
# задаем горизонт
HORIZON = 16
# задаем стартовую дату
start_date = '2015-01-01'

In [3]:
# загружаем исторический набор
train = pd.read_csv(
    DATA_PREFIX.joinpath('train.csv'),
    parse_dates=['date'], 
    infer_datetime_format=True,
)
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [4]:
# загружаем набор новых данных
test = pd.read_csv(
    DATA_PREFIX.joinpath('test.csv'), 
    parse_dates=["date"], 
    infer_datetime_format=True,
)
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


# Создание объекта `TSDataset`

In [5]:
# функция подготовки исторического набора 
# и набора с экзогенными переменными
def prepare_df(df, start_date=None, num_segments=None):
    df = df.drop(columns=['onpromotion', 'id'])
    # создаем сегменты
    df['segment'] = [f'{x}::{y}' for x, y in zip(df['store_nbr'], df['family'])]
    # переименовываем столбцы в соответствии с требованиями ETNA
    df.rename(columns={
        'date': 'timestamp', 
        'sales': 'target', 
    }, inplace=True)
    
    # если стартовая дата не задана, используем ранее
    # созданную стартовую дату
    if start_date is not None:
        df = df[df['timestamp'] >= start_date]
    
    # если количество сегментов задано, формируем сегменты на основе 
    # первых num_segments уникальных значений столбца segments
    if num_segments is not None:
        segments = df['segment'].unique()[:num_segments]
        df = df[df['segment'].isin(segments)]
    
    # формируем датафрейм из 3 обязательных столбцов
    df = df[['timestamp', 'segment', 'target']]
    # выполняем логарифмирование зависимой переменной
    df['target'] = np.log1p(df['target'])
    
    # стартовая дата
    start_date = df['timestamp'].min()
    # последняя дата: последняя дата исторического набора + горизонт
    end_date = df['timestamp'].max() + pd.Timedelta(HORIZON, 'D')
    # создаем индекс дат
    timestamp = pd.date_range(start=start_date, end=end_date, freq='D')
    # создаем пустой список
    df_exog_list = []
    # создаем датафрейм со столбцом 
    # timestamp на основе индекса дат
    df_exog_segment_template = pd.DataFrame({'timestamp': timestamp})
    # формируем датафрейм с экзогенными переменными
    for segment in tqdm(df['segment'].unique()):
        df_exog_segment = df_exog_segment_template.copy()
        df_exog_segment['segment'] = segment
        df_exog_segment['regressor_store_nbr'] = segment.split('::')[0]
        df_exog_segment['regressor_family'] = segment.split('::')[1]
        df_exog_list.append(df_exog_segment)
    df_exog = pd.concat(df_exog_list, ignore_index=True)
    # признакам, которые хотим обрабатывать, как
    # категориальные, присваиваем тип category
    df_exog['regressor_store_nbr'] = df_exog['regressor_store_nbr'].astype(
        'category')
    df_exog['regressor_family'] = df_exog['regressor_family'].astype(
        'category')
    
    return df, df_exog

In [6]:
# создаем исторический набор и набор с экзогенными переменными
train, train_exog = prepare_df(train, start_date=start_date)

  0%|          | 0/1782 [00:00<?, ?it/s]

In [7]:
# смотрим исторический набор
train

,timestamp,segment,target
1297296,2015-01-01,1::AUTOMOTIVE,0.000000
1297297,2015-01-01,1::BABY CARE,0.000000
1297298,2015-01-01,1::BEAUTY,0.000000
1297299,2015-01-01,1::BEVERAGES,0.000000
1297300,2015-01-01,1::BOOKS,0.000000
...,...,...,...
3000883,2017-08-15,9::POULTRY,6.084802
3000884,2017-08-15,9::PREPARED FOODS,5.046987
3000885,2017-08-15,9::PRODUCE,7.791824
3000886,2017-08-15,9::SCHOOL AND OFFICE SUPPLIES,4.804021


In [8]:
# смотрим набор с экзогенными переменными
train_exog

,timestamp,segment,regressor_store_nbr,regressor_family
0,2015-01-01,1::AUTOMOTIVE,1,AUTOMOTIVE
1,2015-01-02,1::AUTOMOTIVE,1,AUTOMOTIVE
2,2015-01-03,1::AUTOMOTIVE,1,AUTOMOTIVE
3,2015-01-04,1::AUTOMOTIVE,1,AUTOMOTIVE
4,2015-01-05,1::AUTOMOTIVE,1,AUTOMOTIVE
...,...,...,...,...
1735663,2017-08-27,9::SEAFOOD,9,SEAFOOD
1735664,2017-08-28,9::SEAFOOD,9,SEAFOOD
1735665,2017-08-29,9::SEAFOOD,9,SEAFOOD
1735666,2017-08-30,9::SEAFOOD,9,SEAFOOD


In [9]:
# загружаем данные о ценах на нефть
oil = pd.read_csv('Data/store_sales/oil.csv', 
                  index_col=['date'], 
                  parse_dates=['date'])
# берем цены на нефть за период с 1 января 
# 2015 года по 15 августа 2017 года
oil = oil[(oil.index >= '2015-01-01') & 
          (oil.index < '2017-08-16')].copy()
# выполняем переиндексацию, у нас появятся выходные дни
idx = pd.date_range(start='2015-01-01', end='2017-08-15')
oil = oil.reindex(idx, fill_value=np.nan)
# создаем столбец с датами на основе индекса
oil['timestamp'] = oil.index
oil

,dcoilwtico,timestamp
2015-01-01,NaN,2015-01-01
2015-01-02,52.72,2015-01-02
2015-01-03,NaN,2015-01-03
2015-01-04,NaN,2015-01-04
2015-01-05,50.05,2015-01-05
...,...,...
2017-08-11,48.81,2017-08-11
2017-08-12,NaN,2017-08-12
2017-08-13,NaN,2017-08-13
2017-08-14,47.59,2017-08-14


In [10]:
# к датафрейму train_exog присоединяем 
# датафрейм с ценами на нефть
train_exog = pd.merge(train_exog, oil, how='left', on='timestamp')
train_exog

,timestamp,segment,regressor_store_nbr,regressor_family,dcoilwtico
0,2015-01-01,1::AUTOMOTIVE,1,AUTOMOTIVE,NaN
1,2015-01-02,1::AUTOMOTIVE,1,AUTOMOTIVE,52.72
2,2015-01-03,1::AUTOMOTIVE,1,AUTOMOTIVE,NaN
3,2015-01-04,1::AUTOMOTIVE,1,AUTOMOTIVE,NaN
4,2015-01-05,1::AUTOMOTIVE,1,AUTOMOTIVE,50.05
...,...,...,...,...,...
1735663,2017-08-27,9::SEAFOOD,9,SEAFOOD,NaN
1735664,2017-08-28,9::SEAFOOD,9,SEAFOOD,NaN
1735665,2017-08-29,9::SEAFOOD,9,SEAFOOD,NaN
1735666,2017-08-30,9::SEAFOOD,9,SEAFOOD,NaN


In [11]:
# создаем объединенный набор
ts = TSDataset(
    df=TSDataset.to_dataset(train), 
    df_exog=TSDataset.to_dataset(train_exog), 
    freq='D', known_future=['regressor_store_nbr', 'regressor_family']
)

In [12]:
# смотрим объединенный набор
ts

segment    10::AUTOMOTIVE                                                 \
feature        dcoilwtico regressor_family regressor_store_nbr    target   
timestamp                                                                  
2015-01-01            NaN       AUTOMOTIVE                  10  0.000000   
2015-01-02          52.72       AUTOMOTIVE                  10  1.386294   
2015-01-03            NaN       AUTOMOTIVE                  10  1.609438   
2015-01-04            NaN       AUTOMOTIVE                  10  0.693147   
2015-01-05          50.05       AUTOMOTIVE                  10  1.945910   
...                   ...              ...                 ...       ...   
2017-08-11          48.81       AUTOMOTIVE                  10  1.098612   
2017-08-12            NaN       AUTOMOTIVE                  10  1.098612   
2017-08-13            NaN       AUTOMOTIVE                  10  0.000000   
2017-08-14          47.59       AUTOMOTIVE                  10  0.693147   
2017-08-15          47.57       AUTOMOTIVE                  10  1.609438   

segment    10::BABY CARE                                              \
feature       dcoilwtico regressor_family regressor_store_nbr target   
timestamp                                                              
2015-01-01           NaN        BABY CARE                  10    0.0   
2015-01-02         52.72        BABY CARE                  10    0.0   
2015-01-03           NaN        BABY CARE                  10    0.0   
2015-01-04           NaN        BABY CARE                  10    0.0   
2015-01-05         50.05        BABY CARE                  10    0.0   
...                  ...              ...                 ...    ...   
2017-08-11         48.81        BABY CARE                  10    0.0   
2017-08-12           NaN        BABY CARE                  10    0.0   
2017-08-13           NaN        BABY CARE                  10    0.0   
2017-08-14         47.59        BABY CARE                  10    0.0   
2017-08-15         47.57        BABY CARE                  10    0.0   

segment    10::BEAUTY                   ...          9::PRODUCE            \
feature    dcoilwtico regressor_family  ... regressor_store_nbr    target   
timestamp                               ...                                 
2015-01-01        NaN           BEAUTY  ...                   9  0.000000   
2015-01-02      52.72           BEAUTY  ...                   9  0.000000   
2015-01-03        NaN           BEAUTY  ...                   9  0.000000   
2015-01-04        NaN           BEAUTY  ...                   9  0.000000   
2015-01-05      50.05           BEAUTY  ...                   9  0.000000   
...               ...              ...  ...                 ...       ...   
2017-08-11      48.81           BEAUTY  ...                   9  7.282127   
2017-08-12        NaN           BEAUTY  ...                   9  7.258598   
2017-08-13        NaN           BEAUTY  ...                   9  7.435206   
2017-08-14      47.59           BEAUTY  ...                   9  7.207434   
2017-08-15      47.57           BEAUTY  ...                   9  7.791824   

segment    9::SCHOOL AND OFFICE SUPPLIES                              \
feature                       dcoilwtico            regressor_family   
timestamp                                                              
2015-01-01                           NaN  SCHOOL AND OFFICE SUPPLIES   
2015-01-02                         52.72  SCHOOL AND OFFICE SUPPLIES   
2015-01-03                           NaN  SCHOOL AND OFFICE SUPPLIES   
2015-01-04                           NaN  SCHOOL AND OFFICE SUPPLIES   
2015-01-05                         50.05  SCHOOL AND OFFICE SUPPLIES   
...                                  ...                         ...   
2017-08-11                         48.81  SCHOOL AND OFFICE SUPPLIES   
2017-08-12                           NaN  SCHOOL AND OFFICE SUPPLIES   
2017-08-13                           NaN  SCHOOL AND OF

In [13]:
num_lags = 50

# задаем список преобразований/признаков
transforms = [
    MinMaxScalerTransform(in_column='target'),
    LagTransform(in_column='target', 
                 lags=[HORIZON + i for i in range(num_lags)],
                 out_column='lag'),
    DateFlagsTransform(
        day_number_in_week=True,
        day_number_in_month=True,
        is_weekend=True,
        out_column='datetime'),
    MeanTransform(in_column='target', window=32, out_column='mean32'),
    TimeSeriesImputerTransform(in_column='dcoilwtico', 
                               strategy='forward_fill', 
                               window=16),
    LagTransform(in_column='dcoilwtico', 
                 lags=[1],
                 out_column='dcoilwtico_lag'),
    MeanTransform(in_column='dcoilwtico_lag_1', 
                  window=16, 
                  out_column='dcoilwtico_mean16_on_lag_1'),
    FilterFeaturesTransform(exclude=['dcoilwtico', 'dcoilwtico_lag_1'])
]

# Перекрестная проверка расширяющимся окном

In [14]:
# создаем модель
model = LGBMMultiSegmentModel(n_estimators=400, 
                              learning_rate=0.08,
                              min_data_in_leaf=80,
                              subsample=0.6)
# передаем в конвейер модель, горизонт 
# и набор преобразований/признаков
pipeline = Pipeline(model=model, 
                    horizon=HORIZON, 
                    transforms=transforms)
# запускаем перекрестную проверку 
# расширяющимся окном
metrics_df, forecast_df, fold_info_df = pipeline.backtest(
    ts=ts, metrics=[MSE()], n_folds=3
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min remaining:    0.0s


[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  3.8min remaining:    0.0s


[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.8min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.8min finished


In [15]:
# смотрим метрики по 3 тестовым выборках 
# для первых 2 сегментов
n_folds = 3
n_segments = 2
n = n_folds * n_segments
metrics_df.head(n)

,segment,MSE,fold_number
0,10::AUTOMOTIVE,0.332154,0
0,10::AUTOMOTIVE,0.468113,1
0,10::AUTOMOTIVE,0.400572,2
1,10::BABY CARE,0.000124,0
1,10::BABY CARE,0.000052,1
1,10::BABY CARE,0.000073,2


In [16]:
# смотрим значение RMSE, усредненное по сегментам
metric = metrics_df['MSE'].mean()**(1/2)
print(f'RMSE: {metric:.4f}')

RMSE: 0.4307


# Обучение на всем историческом наборе

In [17]:
# создаем модель
model = LGBMMultiSegmentModel(n_estimators=400, 
                              learning_rate=0.08, 
                              min_data_in_leaf=80,
                              subsample=0.6)
# передаем в конвейер модель, горизонт 
# и набор преобразований/признаков
pipeline = Pipeline(model=model, horizon=HORIZON, transforms=transforms)

In [18]:
# обучаем конвейер на всем историческом наборе
pipeline.fit(ts=ts);

[LightGBM] [Warning] min_data_in_leaf is set=80, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=80


In [19]:
# получаем набор с прогнозами
forecasted = pipeline.forecast()
forecasted

segment                  10::AUTOMOTIVE                              \
feature    datetime_day_number_in_month datetime_day_number_in_week   
timestamp                                                             
2017-08-16                           16                           2   
2017-08-17                           17                           3   
2017-08-18                           18                           4   
2017-08-19                           19                           5   
2017-08-20                           20                           6   
2017-08-21                           21                           0   
2017-08-22                           22                           1   
2017-08-23                           23                           2   
2017-08-24                           24                           3   
2017-08-25                           25                           4   
2017-08-26                           26                           5   
2017-08-27                           27                           6   
2017-08-28                           28                           0   
2017-08-29                           29                           1   
2017-08-30                           30                           2   
2017-08-31                           31                           3   

segment                                                               \
feature    datetime_is_weekend dcoilwtico dcoilwtico_mean16_on_lag_1   
timestamp                                                              
2017-08-16               False        NaN                  49.056250   
2017-08-17               False        NaN                  48.979333   
2017-08-18               False        NaN                  48.964286   
2017-08-19                True        NaN                  48.915385   
2017-08-20                True        NaN                  48.905833   
2017-08-21               False        NaN                  48.845455   
2017-08-22               False        NaN                  48.773000   
2017-08-23               False        NaN                  48.684444   
2017-08-24               False        NaN                  48.598750   
2017-08-25               False        NaN                  48.531429   
2017-08-26                True        NaN                  48.355000   
2017-08-27                True        NaN                  48.318000   
2017-08-28               False        NaN                  48.195000   
2017-08-29               False        NaN                  47.990000   
2017-08-30               False        NaN                  47.580000   
2017-08-31               False        NaN                  47.570000   

segment                                                       ... 9::SEAFOOD  \
feature       lag_16    lag_17    lag_18    lag_19    lag_20  ...     lag_60   
timestamp                                                     ...              
2017-08-16  0.355418  0.000000  0.000000  0.224244  0.000000  ...   0.787679   
2017-08-17  0.224244  0.355418  0.000000  0.000000  0.224244  ...   0.810536   
2017-08-18  0.000000  0.224244  0.355418  0.000000  0.000000  ...   0.676614   
2017-08-19  0.355418  0.000000  0.224244  0.355418  0.000000  ...   0.656358   
2017-08-20  0.520678  0.355418  0.000000  0.224244  0.355418  ...   0.697126   
2017-08-21  0.355418  0.520678  0.355418  0.000000  0.224244  ...   0.661814   
2017-08-22  0.355418  0.355418  0.520678  0.355418  0.000000  ...   0.537138   
2017-08-23  0.448488  0.355418  0.355418  0.520678  0.355418  ...   0.801038   
2017-08-24  0.000000  0.448488  0.355418  0.355418  0.520678  ...   0.784231   
2017-08-25  0.448488  0.000000  0.448488  0.355418  0.355418  ...   0.625484   
2017-08-26  0.000000  0.448488  0.000000  0.448488  0.355418  ...   0.597001   
2017-08-27  0.355418  0.000000  0.448488  0.000000  0.448488  ...   0.621126   
2017-08-28  0.355418  0.355418  0.000000  0.448488  0.000000  ...   0.588281   
2

# Формирование файла посылки

In [20]:
# загружаем файл посылки
sample_submission = pd.read_csv(DATA_PREFIX.joinpath('sample_submission.csv'))
# выводим первые 5 наблюдений посылки
sample_submission.head()

,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


In [21]:
# выводим первые 5 наблюдений набора новых данных
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [22]:
# берем прогнозы
forecasted_target = forecasted[:, :, 'target']
# создаем серию из столбца sales файла посылки
sales = sample_submission['sales'].copy()
# записываем прогнозы в только что созданную серию
for i, row in tqdm(test.iterrows(), total=test.shape[0]):
    date = row['date']
    segment = f"{row['store_nbr']}::{row['family']}"
    try:
        target = forecasted_target.loc[date, segment].item()
        sales.iloc[i] = target
    except IndexError:
        pass

  0%|          | 0/28512 [00:00<?, ?it/s]

In [23]:
# смотрим статистики серии
sales.describe()

count    28512.000000
mean         3.602794
std          2.488286
min         -0.063615
25%          1.524668
50%          3.359068
75%          5.560898
max          9.759106
Name: sales, dtype: float64

In [24]:
# серия с прогнозами становится столбцом sales файла посылки
# если есть отрицательные продажи, берем 0, и выполняем экспоненцирование
# (поскольку ранее логарифмировали зависимую переменную)
sample_submission['sales'] = np.expm1(np.maximum(sales, 0))

In [25]:
# смотрим первые 5 наблюдений файла посылки
sample_submission.head()

,id,sales
0,3000888,3.860037
1,3000889,0.006943
2,3000890,3.357213
3,3000891,2450.652242
4,3000892,0.124380


In [26]:
# записываем посылку в виде CSV-файла
sample_submission.to_csv('etna_store_sales_non_regressors_and_regressors.csv', 
                         index=False)